## Deliverable 2. Create a Customer Travel Destinations Map.

In [94]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

#pulling config file from main directory 1 level up 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append("../")

# Import the API key.
from config import gmap_key


# Configure gmaps API key
gmaps.configure(api_key=gmap_key)

In [95]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Grindavik,IS,63.8424,-22.4338,48.67,100,100,21.85,overcast clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,82.96,75,87,12.77,overcast clouds
2,2,Kapaa,US,22.0752,-159.3190,81.48,77,82,5.99,broken clouds
3,3,Panalingaan,PH,8.7842,117.4231,79.75,82,84,1.63,broken clouds
4,4,Saint George,US,37.1041,-113.5841,99.43,6,0,9.22,clear sky


In [96]:
# 2. Prompt the user to enter minimum and maximum temperature criteria. Putting in function so it can be reuseable

min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))



What is your desired minimum temperature for your trip? 70
What is your desired maximum temperature for your trip? 100


In [60]:
print(min_temp)
print(max_temp)

70.0
100.0


In [61]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

user_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]

user_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Hithadhoo,MV,-0.6000,73.0833,82.96,75,87,12.77,overcast clouds
2,2,Kapaa,US,22.0752,-159.3190,81.48,77,82,5.99,broken clouds
3,3,Panalingaan,PH,8.7842,117.4231,79.75,82,84,1.63,broken clouds
4,4,Saint George,US,37.1041,-113.5841,99.43,6,0,9.22,clear sky
5,5,Hilo,US,19.7297,-155.0900,85.28,84,12,8.01,few clouds


In [62]:
#see how many results matched user input temp
user_cities_df.count()

City_ID        287
City           287
Country        287
Lat            287
Lng            287
Max Temp       287
Humidity       287
Cloudiness     287
Wind Speed     287
Description    287
dtype: int64

In [63]:
# 4a. Determine if there are any empty rows.
user_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [64]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = user_cities_df.dropna()

clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Hithadhoo,MV,-0.6000,73.0833,82.96,75,87,12.77,overcast clouds
2,2,Kapaa,US,22.0752,-159.3190,81.48,77,82,5.99,broken clouds
3,3,Panalingaan,PH,8.7842,117.4231,79.75,82,84,1.63,broken clouds
4,4,Saint George,US,37.1041,-113.5841,99.43,6,0,9.22,clear sky
5,5,Hilo,US,19.7297,-155.0900,85.28,84,12,8.01,few clouds


In [65]:
#confirmed no rows were dropped due to NA values 

len(clean_df)

287

In [66]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name" and rename "Description" to "Current Description"
hotel_df["Current Description"] = hotel_df["Description"]
hotel_df["Hotel Name"] = ""

hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Current Description,Hotel Name
1,Hithadhoo,MV,82.96,overcast clouds,-0.6000,73.0833,overcast clouds,
2,Kapaa,US,81.48,broken clouds,22.0752,-159.3190,broken clouds,
3,Panalingaan,PH,79.75,broken clouds,8.7842,117.4231,broken clouds,
4,Saint George,US,99.43,clear sky,37.1041,-113.5841,clear sky,
5,Hilo,US,85.28,few clouds,19.7297,-155.0900,few clouds,
7,Sal Rei,CV,72.16,broken clouds,16.1833,-22.9167,broken clouds,
11,Rikitea,PF,76.73,broken clouds,-23.1203,-134.9692,broken clouds,
23,Cayenne,GF,77.09,heavy intensity rain,4.9333,-52.3333,heavy intensity rain,
28,Albany,US,74.25,scattered clouds,42.6001,-73.9662,scattered clouds,
29,Atuona,PF,78.39,light rain,-9.8000,-139.0333,light rain,


In [70]:
# Check GMAP API Call 
# Target city
target_city = "Boise, Idaho"

# Build the endpoint URL
target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_city, gmap_key)

# Run a request to endpoint and convert result to json
geo_data = requests.get(target_url).json()

# Print the json
geo_data




{'results': [{'address_components': [{'long_name': 'Boise',
     'short_name': 'Boise',
     'types': ['locality', 'political']},
    {'long_name': 'Ada County',
     'short_name': 'Ada County',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'Idaho',
     'short_name': 'ID',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'United States',
     'short_name': 'US',
     'types': ['country', 'political']}],
   'formatted_address': 'Boise, ID, USA',
   'geometry': {'bounds': {'northeast': {'lat': 43.6898951,
      'lng': -116.1019091},
     'southwest': {'lat': 43.511717, 'lng': -116.3658869}},
    'location': {'lat': 43.6150186, 'lng': -116.2023137},
    'location_type': 'APPROXIMATE',
    'viewport': {'northeast': {'lat': 43.6898951, 'lng': -116.1019091},
     'southwest': {'lat': 43.511717, 'lng': -116.3658869}}},
   'place_id': 'ChIJnbRH6XLxrlQRm51nNpuYW5o',
   'types': ['locality', 'political']}],
 'status': 'OK'}

In [81]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gmap_key
}

# 6b. Iterate through the hotel DataFrame. "iterrows()" will loop thru the rows in the hotel_df which is a copy of "WeatherPy_Database.csv"
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame. Adding location "(lat, lng)" to the params obj required by gmap 
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. The request method allows the url and query params of the api as seperate arguments
    hotels = requests.get(base_url, params=params).json()
#     print(hotels)
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        print("Hotel Found")  
        hotel_df.loc[index, 'Hotel Name'] = hotels["results"][0]['name']
    except(IndexError):
        print("Cannot find hotel...skipping.")  
    
        

Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Cannot find hotel...skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Cannot find hotel...skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Cannot find hotel...skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Cannot find hotel...skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Cannot find hotel...skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Foun

In [82]:
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Current Description,Hotel Name
1,Hithadhoo,MV,82.96,overcast clouds,-0.6000,73.0833,overcast clouds,Scoop Guest House
2,Kapaa,US,81.48,broken clouds,22.0752,-159.3190,broken clouds,Sheraton Kauai Coconut Beach Resort
3,Panalingaan,PH,79.75,broken clouds,8.7842,117.4231,broken clouds,I Love Panalingaan View Deck
4,Saint George,US,99.43,clear sky,37.1041,-113.5841,clear sky,Best Western Plus Abbey Inn
5,Hilo,US,85.28,few clouds,19.7297,-155.0900,few clouds,Hilo Hawaiian Hotel


In [85]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Description            0
Lat                    0
Lng                    0
Current Description    0
Hotel Name             0
dtype: int64

In [90]:
# 7. Drop the rows where there is no Hotel Name.
no_hotel_names = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = no_hotel_names.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Current Description,Hotel Name
1,Hithadhoo,MV,82.96,overcast clouds,-0.6000,73.0833,overcast clouds,Scoop Guest House
2,Kapaa,US,81.48,broken clouds,22.0752,-159.3190,broken clouds,Sheraton Kauai Coconut Beach Resort
3,Panalingaan,PH,79.75,broken clouds,8.7842,117.4231,broken clouds,I Love Panalingaan View Deck
4,Saint George,US,99.43,clear sky,37.1041,-113.5841,clear sky,Best Western Plus Abbey Inn
5,Hilo,US,85.28,few clouds,19.7297,-155.0900,few clouds,Hilo Hawaiian Hotel


In [91]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [92]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [93]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))